# Projeto 2 - Ciência dos Dados

**Nome**: Jonas da Silva Lopes

**Nome**: William Silva


# Classificador automático de sentimento

## Preparando o ambiente no jupyter:

In [22]:
%%capture
!pip install tweepy
!pip install emoji

In [24]:
import tweepy
import emoji
import math
import os.path
import pandas as pd
import json
import re 
import functools
import operator
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@William48253649***

In [25]:
#Identificador da conta no twitter: @William48253649

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [26]:
#Produto escolhido
produto = 'Nubank'

#Quantidade mínima de mensagens capturadas
n = 500
#Quantidade mínima de mensagens para a base de treinamento
t = 300

#Filtro de língua
lang = 'pt'

Capturando os dados do twitter:

In [27]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower()) #Lower para deixar tudo minúsculo e facilitar a comparação
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [62]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])}).set()
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])}).set()
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Após realizar a classificação manual das mensagens, como irrelevante (0) ou relevante (1), partimos para  a mudança desses valores para algo mais palpável para a análise. Com isso, fazemos a alteração que pode ser vista abaixo.

Mas, para realizar isso, primeiro foi necessário determinar critérios para a classificação:

- Mencionar o produto;
- A menção ao produto deve ser acompanhada de uma opinião;
- A opinião pode ser demonstrada na forma de indagações, reclamações, pode envolver sarcasmo, elogios e sugestões sobre serviços;
- A opinião afirmada deve ser clara;
- Emoctions também representam opiniões.

In [66]:
mensagens = pd.read_excel("Nubank.xlsx")
mensagens.Relevância = mensagens.Relevância.astype('category')
mensagens.Relevância.cat.categories = ('Irrelevante', 'Relevante')


relevante = mensagens[mensagens.Relevância=="Relevante"]
irrelevante = mensagens[mensagens.Relevância=="Irrelevante"]

total = len(relevante) + len(irrelevante)

print("A quantidade de cada cada um é: \n\n", mensagens.Relevância.value_counts())

A quantidade de cada cada um é: 

 Irrelevante    184
Relevante      116
Name: Relevância, dtype: int64


In [30]:
'''A Partir dos dados obtidos, nota-se que:
        p(relevante) = 116/300
        p(irrelevante) = 184/300
'''
p_relev = 116/300*100
print("A probabilidade de ser relevante é", p_relev, "%")

p_irrelev = 184/300*100
print("A probabilidade de ser irrelevante é", p_irrelev, "%")

A probabilidade de ser relevante é 38.666666666666664 %
A probabilidade de ser irrelevante é 61.33333333333333 %


In [93]:
''' Função que troca pontuação por espaço '''
def cleanup(text):
    punctuation = '[!\-.:?;/,|@"\'()]'
    pattern = re.compile(punctuation)
    # Abaixo, determina que se troca por espaço
    text_subbed = re.sub(pattern, ' ', text)
    
    emoji_dividir = emoji.get_emoji_regexp().split(text_subbed)
    espaco_dividir = [substr.split() for substr in emoji_dividir]
    split = functools.reduce(operator.concat, espaco_dividir)
    
    return split

# Usando a função apply para fazer a limpeza nas mensagens
nubank_relev = relevante.Treinamento.apply(cleanup)
nubank_irrelev = irrelevante.Treinamento.apply(cleanup)

# Pegando as palavras da lista de Relevantes para contar
def pega_text(texto):
    # Pega as palavras para colocar numa lista
    lista = []
    i = 0
    for linha in texto:
        if len(linha) > 1:
            while i < len(linha):
                lista.append(linha[i])
                i += 1  
            i = 0
        elif len(linha) != 1 and (len(linha)-1) != 1:
            a = texto
            return a 
    return lista

### Frequência Absoluta

Relevantes

In [32]:
# Frequência dos Relevantes
lista_nubank_relev = pd.DataFrame(pega_text(nubank_relev))
lista_nubank_relev[0].value_counts()

nubank          122
o                58
de               54
e                52
eu               47
que              45
a                35
meu              34
me               29
https            28
co               28
t                28
um               27
não              26
é                25
do               23
cartão           23
no               20
uma              16
pra              15
q                15
na               15
to               14
com              14
só               13
mais             12
limite           12
tem              11
nunca            11
da               11
               ... 
relata            1
cmg               1
continuar         1
#nubank           1
chance            1
simplesmente      1
samba             1
qq                1
apareceu          1
função            1
fechamento        1
tornam            1
curioso           1
crystal           1
futebol           1
for               1
gastei            1
lembrando         1
digital           1


Irrelevantes

In [33]:
# Frequência dos Irrelevantes
lista_nubank_irrelev = pd.DataFrame(pega_text(nubank_irrelev))
lista_nubank_irrelev[0].value_counts()
lista_nubank_irrelev[0].value_counts()

nubank           154
o                 71
de                67
e                 62
que               60
co                58
https             58
t                 58
a                 57
não               52
é                 38
um                36
eu                34
da                33
com               33
do                31
se                30
gente             29
tem               28
rt                28
meu               26
pra               26
no                25
em                25
cartão            24
você              24
na                24
só                22
conta             22
uma               22
                ... 
comprovante        1
cagaço             1
douglas_chgs       1
7wgummaqki         1
garantido          1
ajudem             1
torcida            1
resolveu           1
interferir         1
torrar             1
medo               1
entrega            1
olhada             1
diminuírem         1
esperamos          1
clona              1
pzuhzm8rb0   

### Frequência Relativa

Relevante

In [34]:
lista_nubank_relev[0].value_counts(True)

nubank          0.056092
o               0.026667
de              0.024828
e               0.023908
eu              0.021609
que             0.020690
a               0.016092
meu             0.015632
me              0.013333
https           0.012874
co              0.012874
t               0.012874
um              0.012414
não             0.011954
é               0.011494
do              0.010575
cartão          0.010575
no              0.009195
uma             0.007356
pra             0.006897
q               0.006897
na              0.006897
to              0.006437
com             0.006437
só              0.005977
mais            0.005517
limite          0.005517
tem             0.005057
nunca           0.005057
da              0.005057
                  ...   
relata          0.000460
cmg             0.000460
continuar       0.000460
#nubank         0.000460
chance          0.000460
simplesmente    0.000460
samba           0.000460
qq              0.000460
apareceu        0.000460


Irrelevante

In [35]:
lista_nubank_irrelev[0].value_counts(True)

nubank           0.045441
o                0.020950
de               0.019770
e                0.018294
que              0.017704
co               0.017114
https            0.017114
t                0.017114
a                0.016819
não              0.015344
é                0.011213
um               0.010623
eu               0.010032
da               0.009737
com              0.009737
do               0.009147
se               0.008852
gente            0.008557
tem              0.008262
rt               0.008262
meu              0.007672
pra              0.007672
no               0.007377
em               0.007377
cartão           0.007082
você             0.007082
na               0.007082
só               0.006492
conta            0.006492
uma              0.006492
                   ...   
comprovante      0.000295
cagaço           0.000295
douglas_chgs     0.000295
7wgummaqki       0.000295
garantido        0.000295
ajudem           0.000295
torcida          0.000295
resolveu    

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, estamos ensinando nosso treinador a identificar as probabilidades, para posteriormente fazer a identificação e a leitura. 

#### Relevantes

In [36]:
'''Fazendo dicionários, para serem utilizados com objetivos na função'''

linhas_mens = mensagens.Treinamento.apply(cleanup)[0]

# Dicionário que guarda, dos relevantes, as palavras e quantas vezes apareceu
lista_counts_relevante = (lista_nubank_relev[0].value_counts()).to_dict()

# Dicionário que guarda a quantidade de relevantes e irrelevantes
dicionario_counts_relevancia = (mensagens.Relevância.value_counts()).to_dict()

In [144]:
# Função que calcula as probabilidade de ser relevante
def probabilidade_do_naive(lista_counts_relevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_relev):
    verossimilhança = 1
    for word in linhas_mens:
        for key, values in lista_counts_relevante.items():
            if key == word:
                verossimilhança *= (values+1)/len(lista_nubank_relev)
    
    for rel, v in dicionario_counts_relevancia.items():
        if rel == "Relevante":
            total_rel = v
    priori = total_rel/total
    
    return (('{0:.2f}').format(math.log10(verossimilhança * priori)))

print(probabilidade_do_naive(lista_counts_relevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_relev))

-94.33


In [148]:
probs_relevante = [0]*300

i = 0
for h in probs_relevante:
    linhas_mens = mensagens.Treinamento.apply(cleanup)[i]
    probs_relevante[i] = probabilidade_do_naive(lista_counts_relevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_relev)
    i += 1
probs_relevante

['-94.33',
 '-53.34',
 '-11.67',
 '-15.11',
 '-15.79',
 '-38.20',
 '-18.60',
 '-46.38',
 '-22.95',
 '-37.96',
 '-16.50',
 '-23.07',
 '-49.40',
 '-36.64',
 '-58.45',
 '-25.09',
 '-17.15',
 '-45.56',
 '-15.92',
 '-62.50',
 '-38.70',
 '-18.43',
 '-65.57',
 '-59.22',
 '-25.27',
 '-113.98',
 '-44.09',
 '-17.43',
 '-31.04',
 '-9.51',
 '-72.40',
 '-69.71',
 '-18.43',
 '-47.93',
 '-87.25',
 '-5.94',
 '-22.34',
 '-33.79',
 '-43.73',
 '-27.15',
 '-41.88',
 '-25.21',
 '-21.75',
 '-14.21',
 '-13.21',
 '-71.64',
 '-26.48',
 '-24.66',
 '-43.37',
 '-13.99',
 '-17.73',
 '-13.89',
 '-20.42',
 '-21.01',
 '-14.96',
 '-18.38',
 '-51.91',
 '-40.46',
 '-52.46',
 '-77.59',
 '-91.46',
 '-33.63',
 '-16.22',
 '-36.86',
 '-24.35',
 '-27.61',
 '-34.69',
 '-37.43',
 '-38.70',
 '-55.27',
 '-1.66',
 '-20.54',
 '-42.72',
 '-1.66',
 '-27.81',
 '-22.01',
 '-3.85',
 '-63.67',
 '-40.04',
 '-13.64',
 '-20.21',
 '-17.11',
 '-10.48',
 '-124.38',
 '-17.95',
 '-14.14',
 '-19.13',
 '-46.20',
 '-38.70',
 '-34.77',
 '-11.98',
 '

#### Irrelevantes

In [149]:
# Dicionário que guarda, dos irrelevantes, as palavras e quantas vezes apareceu
lista_counts_irrelevante = (lista_nubank_irrelev[0].value_counts()).to_dict()

In [150]:
# Função que calcula as probabilidade de ser irrelevante
def probabilidade_do_naive_irr(lista_counts_irrelevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_irrelev):
    verossimilhança = 1
    # Identificando a quantidade e calculando a probabilidade de cada palavra do tweet
    for word in linhas_mens:
        for key, values in lista_counts_irrelevante.items():
            if key == word:
                verossimilhança *= (values+1) / len(lista_nubank_irrelev)
    
    # Pegando a probabilidade e dividindo pelo total, usando Naive Bayes
    for rel, v in dicionario_counts_relevancia.items():
        if rel == "Irrelevante":
            total_rel = v
    priori = total_rel/total
    
    return (('{0:.2f}').format(math.log10(verossimilhança * priori)))

In [170]:
probs_irrelevante = [0]*300

i = 0
for h in probs_irrelevante:
    linhas_mens = mensagens.Treinamento.apply(cleanup)[i]
    probs_irrelevante[i] = probabilidade_do_naive_irr(lista_counts_irrelevante, dicionario_counts_relevancia, total, linhas_mens, lista_nubank_irrelev)
    i += 1
#probs_irrelevante

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [177]:
df_teste = pd.ExcelFile("Nubank.xlsx")
df_teste1 = pd.read_excel(df_teste, 'Teste')

#### Criando os argumentos que entraram na função criada

Número de tweets relevantes e irrelevantes e seu total

In [188]:
df_teste1.Relevância = mensagens.Relevância.astype('category')
df_teste1.Relevância.cat.categories = ('Irrelevante', 'Relevante')

relevante_teste = df_teste1[df_teste1.Relevância=="Relevante"]
irrelevante_teste = df_teste1[df_teste1.Relevância=="Irrelevante"]

total_teste = len(relevante_teste) + len(irrelevante_teste)

print("A quantidade de cada cada um é: \n\n", df_teste1.Relevância.value_counts())
dicionario_counts_relevância1 = (df_teste1.Relevância.value_counts()).to_dict()
dicionario_counts_relevância1

A quantidade de cada cada um é: 

 Irrelevante    121
Relevante       79
Name: Relevância, dtype: int64


{'Irrelevante': 121, 'Relevante': 79}

#### Relevante

Frequência absoluta das palavras nos tweets relevantes e transformando em dicionário

In [187]:
# Usando a função apply para fazer a limpeza nas mensagens
df_teste2_rel = relevante_teste.Teste.apply(cleanup)
df_teste2_rel = pd.DataFrame(pega_text(df_teste2_rel))

dict_teste_counts_rel = (df_teste2_rel[0].value_counts()).to_dict()
total_teste_rel = df_teste2_rel

#### Irrelevante

Frequência absoluta das palavras nos tweets irrelevantes e tranformando em dicionário

In [162]:
df_teste2_irrel = pd.DataFrame(pega_text(irrelevante_teste.Teste.apply(cleanup)))
dict_teste_counts_irrel = (df_teste2_irrel[0].value_counts()).to_dict()
total_teste_irrel = df_teste2_irrel


In [167]:
linhas_mens_teste = df_teste1.Teste.apply(cleanup)[0]

In [168]:
print(probabilidade_do_naive_irr(dict_teste_counts_rel, dicionario_counts_relevância1, total, linhas_mens_teste, total_teste_rel))

-9.75


#### Relevante

In [193]:
probs_relevante2 = [0]*200

i = 0
for elemento in probs_relevante2:
    linhas_mens_teste = df_teste1.Teste.apply(cleanup)[i]
    probs_relevante2[i] = probabilidade_do_naive(dict_teste_counts_rel, dicionario_counts_relevância1, total_teste, linhas_mens_teste, df_teste2_rel)
    i += 1



#### Irrelevante

In [197]:
probs_irrelevante2 = [0]*200

i = 0
for elemento in probs_irrelevante2:
    linhas_mens_teste = df_teste1.Teste.apply(cleanup)[i]
    probs_irrelevante2[i] = probabilidade_do_naive_irr(dict_teste_counts_irrel, dicionario_counts_relevância1, total_teste, linhas_mens_teste, df_teste2_irrel)
    i += 1

#### Comparação

In [216]:
def comparation(relevante, irrelevante):
    i = 0 
    result = []
    for e in relevante:
        if relevante[i] > irrelevante[i]:
            result.append("Relevante")
        elif relevante[i] < irrelevante[i]:
            result.append("Irrelevante")
        # É muito improvável que ocorra. Porém, se ocorrer, consideramos que é irrelevante
        else:
            result.append(0)
        i+=1    
        
    return result

In [220]:
resultado_das_relevancias = comparation(probs_relevante2, probs_irrelevante2)

In [223]:
serie = []
for linha in df_teste1.Relevância:
    serie.append(linha)

def comparação_com_classificados(relevancias_comparativas, classificados):
    i = 0 
    contador1 = 0
    contador2 = 0
    comparação_final = []
    for e in classificados:
        if relevancias_comparativas[i] == classificados[i]:
            comparação_final.append(True)
            contador1 += 1
        else:
            comparação_final.append(False)
            contador2 += 1
        i+=1
            
    print(contador1, contador2)    
    
    return comparação_final

comparação_com_classificados(resultado_das_relevancias, serie)




161 39


[True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**